In [ ]:
# ETL DEL DATASET OUTPUT_STEAM_GAMES

#Se desarrolla la extracción, transformación y carga de output_steam_games.json

In [1]:
# IMPORTACIONES

import pandas as pd 
import json
import re

In [2]:
# FUNCIONES IMPORTANTES

# Para verificar tipos de datos y porcentajes
def verificar_tipo_dato(dataframe):
    diccionario = {'nombre_campo': [], 'tipo_dato': [], '%_no_nulos': [], '%_nulos': [], 'nulos': []}
    for columna in dataframe.columns:
        porcentaje_no_nulos = (dataframe[columna].count()/len(dataframe))*100
        diccionario['nombre_campo'].append(columna)
        diccionario['tipo_dato'].append(dataframe[columna].apply(type).unique())
        diccionario['%_no_nulos'].append(round(porcentaje_no_nulos,2))
        diccionario['%_nulos'].append(round(100-porcentaje_no_nulos,2))
        diccionario['nulos'].append(dataframe[columna].isnull().sum())
    
    df_info = pd.DataFrame(diccionario)
    return df_info





# Para ver la existencia de duplicados
def verificar_duplicados_columna(df, columna):
    lista_vacia = []
    filas_duplicadas = df[df.duplicated(subset=columna, keep=False)]
    for elemento in df[columna]:
        if len(filas_duplicadas) == len(lista_vacia):
            return 'No hay elementos duplicados'
        else:
            return filas_duplicadas.sort_values(by=columna, ascending=True)




# Para obtener año de lanzamiento
def obtener_año_lanzamiento(fecha):
    if pd.notna(fecha):
        if re.match(r'^\d{4}-\d{2}-\d{2}$', fecha):
            return fecha.split('-')[0]
    return 'Dato no disponible'




# Para reemplazar a float
def reemplazar_a_float(value):
    if pd.isna(value):
        return 0.0
    try:
        float_value = float(value)
        return float_value
    except:
        return 0.0

In [3]:
# EXTRACCION DE DATOS

# Se extraen los datos de los JSON y se les convierte en dataframe
archivo_games = 'C:\\Users\\cquir\\OneDrive\\Escritorio\\Data Science SH\\Proyecto Individual 1\\bases de datos\\output_steam_games.json'

# Se lee el dataset por linea
filas_games = []
with open(archivo_games) as f:
    for line in f.readlines():
        data = json.loads(line)
        filas_games.append(data)
dfgames = pd.DataFrame(filas_games)
dfgames

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [4]:
# Inicialmente, se observan filas que presentan todos sus valores nulos, por lo que procedemos a eliminarlas. Si no tuvieramos esta primera visualización a mano, tendríamos que proceder a buscarlas de igual forma dentro del DataFrame
dfgames = dfgames.dropna(how='all').reset_index(drop=True)

dfgames.shape

(32135, 13)

In [5]:
#  Con funcion predeterminada se revisan los tipos de datos y la cantidad de nulos presentes
verificar_tipo_dato(dfgames)

# Luego de esta operación, aún quedan filas con valores nulos

,nombre_campo,tipo_dato,%_no_nulos,%_nulos,nulos
0,publisher,"[<class 'str'>, <class 'float'>]",74.94,25.06,8052
1,genres,"[<class 'list'>, <class 'float'>]",89.78,10.22,3283
2,app_name,"[<class 'str'>, <class 'float'>]",99.99,0.01,2
3,title,"[<class 'str'>, <class 'float'>]",93.62,6.38,2050
4,url,[<class 'str'>],100.00,0.00,0
5,release_date,"[<class 'str'>, <class 'float'>]",93.57,6.43,2067
6,tags,"[<class 'list'>, <class 'float'>]",99.49,0.51,163
7,reviews_url,"[<class 'str'>, <class 'float'>]",99.99,0.01,2
8,specs,"[<class 'list'>, <class 'float'>]",97.92,2.08,670
9,price,"[<class 'float'>, <class 'str'>]",95.71,4.29,1377


In [6]:
# VERIFICACIÓN DE VALORES DUPLICADOS


# se constata si existen valores duplicados en este caso para la columna más importante, "id"
verificar_duplicados_columna(dfgames, 'id')

# se observan 2 id duplicados ("612880" y NaN). En primera instancia, podemos borrar el id 612880 (registro 14573) ya que son valores idénticos con el registro 13894. Por otro lado, hay que buscar a ver si los registros NaN corresponden al mismo juego. Haremos esto viendo el developer del registro 30961.

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
13894,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
14573,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
74,NaN,NaN,NaN,NaN,http://store.steampowered.com/,NaN,NaN,NaN,NaN,19.99,False,NaN,NaN
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [7]:
# buscamos el developer "Rocksteady Studios,Feral Interactive (Mac)" para ver a cual juego corresponde el valor NaN del registro 30961

dfgames[dfgames['developer'] == 'Rocksteady Studios,Feral Interactive (Mac)']

# de esta forma podemos ver que el id NaN corresponde por nombre de título al id 200260

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
1068,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260/Batma...,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",http://steamcommunity.com/app/200260/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,200260,"Rocksteady Studios,Feral Interactive (Mac)"
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"
31617,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham Asylum Game of the Year Edition,Batman: Arkham Asylum Game of the Year Edition,http://store.steampowered.com/app/35140/Batman...,2010-03-26,"[Action, Batman, Stealth, Adventure, Third Per...",http://steamcommunity.com/app/35140/reviews/?b...,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,35140,"Rocksteady Studios,Feral Interactive (Mac)"


In [8]:
# por lo tanto los índices a eliminar son:
indices_a_eliminar = [14573, 74, 30961]

dfgames = dfgames.drop(indices_a_eliminar)

# verificamos nuevamente duplicados
verificar_duplicados_columna(dfgames, 'id')

'No hay elementos duplicados'

In [9]:
# COLUMNAS A TRANSFORMAR

# Se necesitan transformar las siguientes columnas:
# 'release_date': De esta columna se necesita extraer el año de lanzamiento. Crearemos una funcion para ello llamada "extraer_año_lanzamiento"
# 'price': Trabajaremos con esta columna, sin embargo los datos no están 'homogenizados' (como items "free to play" o promociones que están presentes como texto). Reemplazaremos estos valores por "0"
# 'publisher', 'app_name', 'title', 'developer', donde se observan valores nulos y se decide completarlos con valores "Dato no disponible"
# 'genres' la cual se presenta como una lista de valores, por lo que hay que desanidarlos 
# 'tags', 'specs', 'url' y 'reviews_url' que no se utilizaran ni para la API ni para el modelo de recomendacion

In [10]:
# COLUMNA 'release_date'

# creamos nueva columna 'anio_lanzamiento' y posteriormente eliminamos la columna 'release_date'
dfgames['anio_lanzamiento'] = dfgames['release_date'].apply(obtener_año_lanzamiento)

dfgames = dfgames.drop('release_date', axis=1)
dfgames.head()

,publisher,genres,app_name,title,url,tags,reviews_url,specs,price,early_access,id,developer,anio_lanzamiento
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,2018
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,2017
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,2017
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,Dato no disponible


In [11]:
dfgames['anio_lanzamiento'].unique()

array(['2018', '2017', 'Dato no disponible', '1997', '1998', '2016',
       '2006', '2005', '2003', '2007', '2002', '2000', '1995', '1996',
       '1994', '2001', '1993', '2004', '1999', '2008', '2009', '1992',
       '1989', '2010', '2011', '2013', '2012', '2014', '1983', '1984',
       '2015', '1990', '1988', '1991', '1985', '1982', '1987', '1981',
       '1986', '2021', '2019', '1975', '1970', '1980'], dtype=object)

In [12]:
# COLUMNA 'price'
dfgames['price'] = dfgames['price'].apply(reemplazar_a_float)
dfgames['price'].dtype

dtype('float64')

In [13]:
# COLUMNA 'publisher', 'app_name', 'title', 'developer'

columnas_a_completar = ['publisher', 'app_name', 'title', 'developer']

# se rellenan nulos
dfrelleno = dfgames[columnas_a_completar].fillna('Sin dato disponible')

# Se borran las columnas originales de dfgames y se concatena el relleno con todo el dataframe. 
dfgames = pd.concat([dfgames.drop(columnas_a_completar, axis=1), dfrelleno], axis=1)
dfgames.head()

,genres,url,tags,reviews_url,specs,price,early_access,id,anio_lanzamiento,publisher,app_name,title,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",http://store.steampowered.com/app/643980/Ironb...,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,643980,2018,"Making Fun, Inc.",Ironbound,Ironbound,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",http://store.steampowered.com/app/670290/Real_...,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,670290,2017,Poolians.com,Real Pool 3D - Poolians,Real Pool 3D - Poolians,Poolians.com
3,"[Action, Adventure, Casual]",http://store.steampowered.com/app/767400/2222/,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,2017,彼岸领域,弹炸人2222,弹炸人2222,彼岸领域
4,NaN,http://store.steampowered.com/app/773570/Log_C...,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,Dato no disponible,Sin dato disponible,Log Challenge,Sin dato disponible,Sin dato disponible


In [14]:
# COLUMNA 'genres'

# esta columna presenta datos anidados en una lista, por lo que necesitamos desanidarlos y hacer una fila con cada genero. Para eso utilizaremos la funcion predeterminada "explode"

dfgames = dfgames.explode('genres')
dfgames = dfgames.dropna(subset='genres')
dfgames.head()

,genres,url,tags,reviews_url,specs,price,early_access,id,anio_lanzamiento,publisher,app_name,title,developer
0,Action,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Casual,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Indie,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Simulation,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Strategy,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro


In [15]:
# COLUMNAS 'tags', 'specs', 'url' y 'reviews_url'

dfgames = dfgames.drop(['tags', 'specs', 'url', 'reviews_url'], axis=1)
dfgames.columns

Index(['genres', 'price', 'early_access', 'id', 'anio_lanzamiento',
       'publisher', 'app_name', 'title', 'developer'],
      dtype='object')

In [16]:
# se verifican los datos y se observa si quedan nulos restantes
verificar_tipo_dato(dfgames)

,nombre_campo,tipo_dato,%_no_nulos,%_nulos,nulos
0,genres,[<class 'str'>],100.0,0.0,0
1,price,[<class 'float'>],100.0,0.0,0
2,early_access,[<class 'bool'>],100.0,0.0,0
3,id,[<class 'str'>],100.0,0.0,0
4,anio_lanzamiento,[<class 'str'>],100.0,0.0,0
5,publisher,[<class 'str'>],100.0,0.0,0
6,app_name,[<class 'str'>],100.0,0.0,0
7,title,[<class 'str'>],100.0,0.0,0
8,developer,[<class 'str'>],100.0,0.0,0


In [17]:
#  GUARDADO DEL DATASET

#Se guarda el dataframe transformado como dfgames_limpio.
dfgames.to_csv(r"C:\Users\cquir\OneDrive\Escritorio\Data Science SH\Proyecto Individual 1\bases de datos\dfgames_limpio.csv", index=None)